# Resnet architecture

In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
from torchvision.models import resnet18
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
import copy
import torch.optim as optim
import os

adaround_model_path = "/content/drive/MyDrive/resnet18_cifar10_adaround.pth"


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = resnet18(weights=None)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()
model.fc = nn.Linear(512, 10)


# Adaround

In [ ]:
adaround = copy.deepcopy(model)

adaround.load_state_dict(torch.load(adaround_model_path))
adaround.eval().to(device)

transform = transforms.Compose([
    transforms.ToTensor()
])

test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = adaround(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'✅ Accuracy of quantized (adaround): {accuracy:.2f}%')

# Model without quantization

In [ ]:
no_quant_model = copy.deepcopy(model)
no_quant_model.load_state_dict(torch.load("/content/drive/MyDrive/resnet18_cifar10_best.pth"))
no_quant_model.eval().to(device)

transform = transforms.Compose([
    transforms.ToTensor()
])  # Bez Normalize, bo nie było tego w treningu

test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = no_quant_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'✅ Accuracy of model: {accuracy:.2f}%')

100%|██████████| 170M/170M [00:18<00:00, 9.12MB/s]


✅ Accuracy of model: 82.71%


# Accuracy with round quanitizing

In [ ]:
round_model = copy.deepcopy(model)
round_model.load_state_dict(torch.load("/content/drive/MyDrive/resnet18_cifar10_best.pth", map_location='cpu'))

# Kwantyzacja przez zwykłe zaokrąglenie wag
quantizer = StaticRoundQuantWrapper(round_model)
quantizer.apply_quant_to_conv_layers()
quantizer.save_model("/content/drive/MyDrive/resnet18_cifar10_rounded.pth")


[Quant] Processing layer: conv1
[Quant] Weights quantized.
[Quant] Processing layer: layer1.0.conv1
[Quant] Weights quantized.
[Quant] Processing layer: layer1.0.conv2
[Quant] Weights quantized.
[Quant] Processing layer: layer1.1.conv1
[Quant] Weights quantized.
[Quant] Processing layer: layer1.1.conv2
[Quant] Weights quantized.
[Quant] Processing layer: layer2.0.conv1
[Quant] Weights quantized.
[Quant] Processing layer: layer2.0.conv2
[Quant] Weights quantized.
[Quant] Processing layer: layer2.0.downsample.0
[Quant] Weights quantized.
[Quant] Processing layer: layer2.1.conv1
[Quant] Weights quantized.
[Quant] Processing layer: layer2.1.conv2
[Quant] Weights quantized.
[Quant] Processing layer: layer3.0.conv1
[Quant] Weights quantized.
[Quant] Processing layer: layer3.0.conv2
[Quant] Weights quantized.
[Quant] Processing layer: layer3.0.downsample.0
[Quant] Weights quantized.
[Quant] Processing layer: layer3.1.conv1
[Quant] Weights quantized.
[Quant] Processing layer: layer3.1.conv2
[Q

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/resnet18_cifar10_rounded.pth"))
model.eval().to(device)

transform = transforms.Compose([
    transforms.ToTensor()
])  # Bez Normalize, bo nie było tego w treningu

test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'✅ Accuracy of quantized (rounded): {accuracy:.2f}%')

✅ Accuracy of quantized (rounded): 82.67%


In [ ]:
#trenowanie ResNet18 na CIFAR-10

# 1. Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Przygotowanie do pracy na urządzeniu: {device}")

# 2. Transforms
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

# 3. Datasets
full_train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
train_size = int(0.9 * len(full_train_dataset))  # 45,000
val_size = len(full_train_dataset) - train_size  # 5,000
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

val_dataset.dataset.transform = transform_test  # Walidacja bez augmentacji

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# 4. DataLoaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

# 5. Model
model = resnet18(weights=None)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()
model.fc = nn.Linear(512, 10)
model = model.to(device)

# 6. Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

# 7. Training with validation
best_val_acc = 0.0
save_path = "/content/drive/MyDrive/resnet18_cifar10_best.pth"

for epoch in range(20):
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            val_correct += predicted.eq(targets).sum().item()
            val_total += targets.size(0)

    val_acc = 100.0 * val_correct / val_total
    print(f"[{epoch+1}/20] Loss: {train_loss/len(train_loader):.4f} | Val Accuracy: {val_acc:.2f}%")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), save_path)
        print(f"✅ Nowy najlepszy model zapisany ({val_acc:.2f}%)")

# 8. Test best model
model.load_state_dict(torch.load(save_path))
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()
        total += targets.size(0)

accuracy = 100 * correct / total
print(f"🎯 Dokładność na zbiorze testowym: {accuracy:.2f}%")
print(f"📦 Model zapisany do: {save_path}")